# Statistical Decision Making
## Task 1

Consider the two-dimensional, discrete random variable $X = [X_1\ X_2]^\top$ subjected to the joint probability density $p_X$ as described in the following table.
<div style="text-align: center">$\begin{array}{c|cc} p_X(X_1, X_2)  & X_2 = 0 & X_2 = 1 \\ \hline\hline X_1 = 0 & 0.4 & 0.3 \\ X_1 = 1 & 0.2 & 0.1\end{array}$</div>   

a) Compute the marginal probability densities $p_{X1}, p_{X2}$ and the conditional probability $P(X_2 = 0|X_1 = 0)$ as well as the expected value $\mathbb{E}[X]$ and the covariance matrix $\mathbb{E}[(X - \mathbb{E}[X])(X - \mathbb{E}[X])^\top]$ by hand.

In [37]:
import numpy as np
p_table = np.array([[0.4,0.3],[0.2,0.1]])
X = np.array([[0,0,1,1],[0,1,0,1]])  #firt:x1  then:x2

In [38]:
#marginal probability
px1 = np.sum(p_table,axis=1)     #行相加
px2 = np.sum(p_table,axis=0)     #列相加
print("marginal probability x1 is [p(x1=0),p(x1=1)]",px1)
print("marginal probability x2 is [p(x2=0),p(x2=1)]",px2)

marginal probability x1 is [p(x1=0),p(x1=1)] [0.7 0.3]
marginal probability x2 is [p(x2=0),p(x2=1)] [0.6 0.4]


In [39]:
#conditional probability
px2equals0condx1equals0 = p_table[0][0]/px1[0]
print('p(x2=0|x1=0)={}'.format(px2equals0condx1equals0))

p(x2=0|x1=0)=0.5714285714285715


In [40]:
#expectation
p_jointdis = p_table.reshape(1,4)  #reshape into row vector
print(p_jointdis)

p_jointdis @ X[0,:] 
p_jointdis @ X[1,:]
Ex = p_jointdis @ X.T
print('the expectation [Ex1 Ex2] is {}'.format(Ex))

[[0.4 0.3 0.2 0.1]]
the expectation [Ex1 Ex2] is [[0.3 0.4]]


<td bgcolor=orange> reshape or ravel  
<br>ravel('C') while keeping row order<br>ravel('F') while keeping column order

In [41]:
#reshape, ravel 
E_X = np.dot(X, p_table.ravel('C')) # ravel('C') while keeping row order
print('Expected value: {}'.format(E_X))
print(E_X.shape)
print(p_table)
print(p_table.ravel('C'))
print(p_table.ravel('F'))

Expected value: [0.3 0.4]
(2,)
[[0.4 0.3]
 [0.2 0.1]]
[0.4 0.3 0.2 0.1]
[0.4 0.2 0.3 0.1]


<td bgcolor=orange> expand_dims is needed so that numpy is able to subtract the vector(2,) from the matrix
<br> covariance matrix as quadratic form

In [42]:
# covariance matrix
#method 1
X_centered = X - np.expand_dims(E_X, axis=1) # expand_dims is needed so that numpy is able to subtract the vector (2,) from the matrix

#method 2 broadcasting
#E_X = E_X[:,None]
#print(E_X.shape)
#X_centered = X - E_X

print("X_centered:\n {}".format(X_centered))
CovX = np.dot(np.dot(X_centered, np.diag(p_table.ravel('C'))), X_centered.T)
print('Covariance matrix:\n {}'.format(CovX))

X_centered:
 [[-0.3 -0.3  0.7  0.7]
 [-0.4  0.6 -0.4  0.6]]
Covariance matrix:
 [[ 0.21 -0.02]
 [-0.02  0.24]]


In [43]:
# broadcasting
m = np.array([[1,2,3,4],[1,2,3,4]])
print(m)
print(m.shape)

n = np.array([[1],[2]])
print(n)
print(n.shape)
print(m-n)

[[1 2 3 4]
 [1 2 3 4]]
(2, 4)
[[1]
 [2]]
(2, 1)
[[ 0  1  2  3]
 [-1  0  1  2]]


In [44]:
np.diag(p_table.ravel('C'))

array([[0.4, 0. , 0. , 0. ],
       [0. , 0.3, 0. , 0. ],
       [0. , 0. , 0.2, 0. ],
       [0. , 0. , 0. , 0.1]])

b) Write a PYTHON function `toyrnd` that expects the positive integer parameter `n` as its input and returns a matrix `X` of size `[2,n]`, containing `n` samples drawn independently from the distribution $p_X$, as its output.

In [45]:
def toyrnd(n):
    Samples = np.random.rand(n)
    for i in range(0,n):
        if Samples[i] > 0.9:
            Samples[i] = 3
        elif Samples[i]>0.7:
            Samples[i] = 2 
        elif Samples[i]>0.4:
            Samples[i] = 1
        else:
            Samples[i] = 0
    Samples_new = np.array([Samples,Samples])
    #print(Samples_new.shape)
    Samples_new[0,:]=Samples_new[0,:]//2
    Samples_new[1,:]=Samples_new[1,:]%2
    return Samples_new

In [62]:
def toyrnd2(n):
    X_out = np.zeros((2,n))
    Q = np.zeros((n,))
    T = np.random.rand(n)  #uniformaly distribution 
     
    # Interpreting [x1, x2] as binary number and Q as its decimal representation
    Q[T>0.4] = 1
    Q[T>0.7] = 2
    Q[T>0.9] = 3
    
    X_out[0] = Q // 2 # floor division
    X_out[1] = Q % 2 # modulus division
    
    return X_out

c) Verify your results in a) by generating `10000` samples with `toyrnd` and computing the respective empirical values.

<td bgcolor=orange>Find the number of desired characteristic
    <br>np.sum(X_observed[0,:]==0)

In [46]:
N = 10000
X_observed = toyrnd(N)
p_X1equals0_empirical = np.float(np.sum(X_observed[0,:]==0))/N
p_X1equals1_empirical = np.float(np.sum(X_observed[0,:]==1))/N
p_X2equals0_empirical = np.float(np.sum(X_observed[1,:]==0))/N
p_X2equals1_empirical = np.float(np.sum(X_observed[1,:]==1))/N
p_x1_empirical = np.array([p_X1equals0_empirical, p_X1equals1_empirical])
p_x2_empirical = np.array([p_X2equals0_empirical, p_X2equals1_empirical])
print('Empirical marginal probability: [p(x1=0), p(x1=1)] = {}'.format(p_x1_empirical))
print('Empirical marginal probability: [p(x2=0), p(x2=1)] = {}'.format(p_x2_empirical))

Empirical marginal probability: [p(x1=0), p(x1=1)] = [0.7013 0.2987]
Empirical marginal probability: [p(x2=0), p(x2=1)] = [0.6019 0.3981]


In [50]:
E_X_empirical = np.sum(X_observed,axis=1)/10000  #row addition
print('Empirical expectation [Ex1,Ex2] = {}'.format(E_X_empirical))
print(E_X_empirical.shape)

Empirical expectation [Ex1,Ex2] = [-2.84217094e-18  1.63424829e-17]
(2,)


In [48]:
X2condX1equals0 = X_observed[1,X_observed[0,:]==0]
print()
X20conX1equals0 = np.float(np.sum(X2condX1equals0==0)/len(X2condX1equals0))
print('Empirical probability: x2=0 under condition x1=0] = {}'.format(X20conX1equals0))


Empirical probability: x2=0 under condition x1=0] = 0.5755026379580779


In [49]:
#X_observed[0,:] = X_observed[0,:] - EX[0]
#X_observed[1,:] = X_observed[1,:] - EX[1]

X_observed = X_observed - np.expand_dims(E_X_empirical, axis=1)
print((X_observed @ X_observed.T)/10000)

[[ 0.20947831 -0.01851247]
 [-0.01851247  0.23961639]]
